In [ ]:
#@title Download App
%tensorflow_version 1.x
import tensorflow as tf
if tf.test.gpu_device_name() != '/device:GPU:0':
  raise SystemError('\n\n------ FOLLOW THIS TO FIX THIS ERROR -------\nYou will need to enable GPU in the notebook settings:\n\n\n1. Navigate to Edit → Notebook Settings\n2. From the Hardware Accelerator drop-down select "GPU"\n3. Restart the notebook')
%cd /content
!rm -r psychoGAN
!git clone https://github.com/AdamSobieszek/psychoGAN
%cd psychoGAN/stylegan2
!gdown https://drive.google.com/u/1/uc?id=1k5hb9fXcq4Mnenvq8utIBXHt5mJWawyP&export=download
%cd ..
!yes | pip install jupyter-dash --quiet
!yes | pip install dash-bootstrap-components --quiet

TensorFlow 1.x selected.
/content
rm: cannot remove 'psychoGAN': No such file or directory
Cloning into 'psychoGAN'...
remote: Enumerating objects: 576, done.
remote: Counting objects: 100% (576/576), done.
remote: Compressing objects: 100% (394/394), done.
remote: Total 576 (delta 302), reused 441 (delta 174), pack-reused 0
Receiving objects: 100% (576/576), 4.49 MiB | 42.94 MiB/s, done.
Resolving deltas: 100% (302/302), done.
/content/psychoGAN/stylegan2
Downloading...
From: https://drive.google.com/u/1/uc?id=1k5hb9fXcq4Mnenvq8utIBXHt5mJWawyP
To: /content/psychoGAN/stylegan2/stylegan2-ffhq-config-f.pkl
382MB [00:06, 60.7MB/s]
/content/psychoGAN
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 1.0MB 37.5MB/s 
     |████████████████████████████████| 3.5MB 50.2MB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
     |████████████████████████████████| 1.8MB 45.5MB/s 
     |████████████████████████████████| 358kB 49.1MB/s 
     |█████

##Import dependecies

In [ ]:
  from jupyter_dash import JupyterDash
import dash_html_components as html
import plotly.express as px
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import json

from skimage import io as skio
import io
import base64
import PIL.Image
import pickle
from time import time
import pandas as pd
import os
import traceback
from main import main 
from generator import generator
import numpy as np
from PIL import Image, ImageDraw
from math import ceil
import numpy as np
from io import BytesIO
import IPython.display

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# app=JupyterDash(__name__, external_stylesheets=external_stylesheets)
df= pd.DataFrame([]) 

In [ ]:
title = html.H1('Face Stimuli Generator',style={'text-align':'center', 'color': '#1d9fad'}, id = "Title")

## Video w zakładce download

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

video_path1 = '/content/psychoGAN/Videos/change_face.mp4'

mp4 = open(video_path1, "rb").read()
data_url1 = "data:video/mp4;base64," + b64encode(mp4).decode()

#
video_path2 = '/content/psychoGAN/Videos/dominance.mp4'

mp4 = open(video_path2, "rb").read()
data_url2 = "data:video/mp4;base64," + b64encode(mp4).decode()
#

video_path3 = '/content/psychoGAN/Videos/trust.mp4'

mp4 = open(video_path3, "rb").read()
data_url3 = "data:video/mp4;base64," + b64encode(mp4).decode()
        
                   
Film1_change_face=  html.Div(
        id='movie1',
        children = [
        html.Video(
            #controls = True,
            id = 'change_face',
            #src = "https://www.w3schools.com/html/mov_bbb.mp4",
            src=data_url1,
            autoPlay=True,
            loop=True,
            title="sample video",
            style={'width':500},#,"position": "fixed",'right':1000,},
            #preload='metadata'

        ),
    ])
      
Film2_dominance=   html.Div(
 
        children = [
        html.Video(
            #controls = True,
            id = 'Dominance',
            #src = "https://www.w3schools.com/html/mov_bbb.mp4",
            src=data_url2,
            autoPlay=True,
            loop=True,
            title="sample video",
            style={'width':500}#,"position": "fixed",'right':1000,},
            #preload='metadata'

        ),
    ])

Film3_trust=   html.Div(
 
        children = [
        html.Video(
            #controls = True,
            id = 'Trust',
            #src = "https://www.w3schools.com/html/mov_bbb.mp4",
            src=data_url3,
            autoPlay=True,
            loop=True,
            title="sample video",
            style={'width':500}#,"position": "fixed",'right':1000,},
            #preload='metadata'

        ),
    ])    

Filmy

In [ ]:
Filmy= html.Div(
         id='film-content-classes',
                   children=[
                         
           html.Video(
            #controls = True,
            id = 'Dominance',
            #src = "https://www.w3schools.com/html/mov_bbb.mp4",
            src=data_url2,
            autoPlay=True,
            loop=True,
            title="sample video",
            style={'width':500},#,"position": "fixed",'right':1000,},
            #preload='metadata'

        ),

            html.Video(
            #controls = True,
            id = 'Trust',
            #src = "https://www.w3schools.com/html/mov_bbb.mp4",
            src=data_url3,
            autoPlay=True,
            loop=True,
            title="sample video",
            style={'width':500},#,"position": "fixed",'right':1000,},
            #preload='metadata'

        ),

        
            html.Video(
            #controls = True,
            id = 'change_face',
            #src = "https://www.w3schools.com/html/mov_bbb.mp4",
            src=data_url1,
            autoPlay=True,
            loop=True,
            title="sample video",
            style={'width':500}#,"position": "fixed",'right':100,},
            #preload='metadata'

        )


    ])

##Instrukcja

In [ ]:
instrukcja= [
        html.H4("Welcome!", className="alert-heading"),
        html.P(
            ["The goal of this application is to generate face stimuli modified on selected traits. ",
            html.Br(),
              "Depending on your needs, you can change how diverse the generated faces will be (truncation), the strength of manipulation, and ho many faces and how many conditions per face you need."]
              ),
        html.Hr(),
        html.P([
            "The first tab 'Generate' has a panel called 'Manipulation settings' that allows you to change the manipulated traits, its strength and number of conditions.",html.Br(),html.Br(),
            "1. The manipulated trait lets you choose between different traits that you can manipulate. However, only the 'dominance' and 'trustworthiness' traits have been obtained from human ratings, and only they have been experimentally checked to be valid manipulations. The results of this experimental validation are showcased below the preview, by displaying the expected difference in this trait's impressions. The differenced in expressed in terms of Standard Deviations, so that you can get an idea of how much did the face shift in the distribution of faces on this particular trait.",html.Br(),
            "2. The strength of the manipulation determines the amount of manipulation on the most manipulated version of the image.",html.Br(),
            "3. The number of conditions determines, whether you want the generator to manipulate the images (0 = no manipulation), but also how many different strengths of manipulation should be generated per face.",html.Br(),html.Br(),
            "Next to this panel you can find 'Stylegan settings' that allows to set how diverse generated faces will be and number of generated images.",html.Br(),
            "Changes in these settings will result in changes in the preview image as well as the final set of generated images. If you want to have another face, just click the 'Change Face' button.",html.Br(),
],className="mb-0",
        ),
        html.Hr(),
        html.P([
            "When you are done with editing settings, click the 'Generate' button to generate the given number of images.",html.Br(),
            "After this process is finished, go to the second tab called 'Download' and click the 'Download Generated Images' button if you want to download generated pictures and their coordinates. To help you decide, the adjacent photo showcases 9 photos from the database you are about to download. The downloading process takes place inside the app, so from your perspective the app will be frozen for the duration of the download, this is not a problem, as the database will have been downloaded when this process finishes.",html.Br(),
            ],className="mb-0",
        ),

    ]

##Pop up przysku Generate

In [ ]:
spiner=      html.Div(
         [
        dbc.Spinner(color="primary", type="grow",),
        dbc.Spinner(color="secondary", type="grow"),
        dbc.Spinner(color="success", type="grow"),
        dbc.Spinner(color="warning", type="grow"),
        dbc.Spinner(color="danger", type="grow"),
        dbc.Spinner(color="info", type="grow"),
        dbc.Spinner(color="light", type="grow"),
        dbc.Spinner(color="dark", type="grow"),
    ]
)
        
ladowarka= html.Div(
    [
        
        dbc.Toast(
            spiner,
            
            
            id="positioned-toast",
            header="Loading the preview...",
            # duration=8000,
            is_open=True,
            dismissable=True,
            #icon="danger",
            # top: 66 positions the toast below the navbar
            style={"position": "fixed", "top": 66, "right": 10, "width": 350},
        ),
    ]
)

##Top buttons i preview


In [ ]:
top_buttons = dbc.Row([               
                                 dbc.Col(dbc.Card(
    dbc.CardBody(
        [dbc.Button(id= 'change_face_button',color='dark',outline=True,size='lg', n_clicks=0, children = 'Change Face', 
                    style={'color':'white','width':'40%', 'border':'2 px black solid', 'height': '50px','text-align':'center', 'marginLeft': '150px', 'marginTop': 2})])
    ,style={"width": "33.7rem"},
    color='#1d9fad',  
    inverse=True,  
    outline=False,  )),
            #Przycisk Generate 
          dbc.Col(dbc.Card(dbc.CardBody(
              html.Div([
                dbc.Button(id= 'generate_button',color='dark',outline=True,size='lg' ,n_clicks=0, children = 'Generate', 
                           style={'color':'white','width':'40%', 'border':'2 px black solid', 'height': '50px','text-align':'center', 'marginLeft': '150px', 'marginTop': 2}),
                  dbc.Modal([
                                  
                      dbc.ModalHeader("Generation in progress... This make take several minutes", id = "header-generation"),
                      dbc.ModalBody(html.Div([
                                  
                                # dcc.Interval(id="progress-interval", n_intervals=0, interval=2500),
                                      # dcc.Loading()
                                      html.P("Please wait. If the process is complete a message will appear on the top of this window. Note this may take several minutes", id = "generate_info"),
                                      dbc.Spinner(id="generate_spiner")
                                              ]
                                                  )),

                                      dbc.ModalFooter(
                                    dbc.Button("Cancel", id="cancel", className="ml-auto")
                                        )],id="modal")
                          ])
                                          
                                            
                                          )
          ,style={"width": "33.7rem"},
    color='#1d9fad',  
    inverse=True,  
    outline=False,))
               ],justify='between')


preview_card = dbc.Card(
    [dbc.CardBody(dbc.Row(dcc.Graph(id = "preview", figure={}), justify="center")
        ),
    ],
    style={"width": "69.5rem", 'marginLeft': '0rem','marginTop': 20},color='#FFFFFF'
)
preview_settings = dbc.CardDeck(
    [
        dbc.Card(
            dbc.CardBody(
                [       html.P(
                        "",
                        className="card-text",
                        id = 'left_estimate',
                        style = {'text-align':'center', 'color':'black'},
                    ),
                ]
            ),style = {'background-color':'#dfe3e6'}
        ),
        dbc.Card(
            dbc.CardBody(
                [       html.P(
                        "Please wait, the preview is loading...",
                        className="card-text",
                        id = "preview-text",
                        style = {'text-align':'center'}
                    ),
                    
                ]
            ),style = {'background-color':'#1d9fad'}
        ),
        dbc.Card(
            dbc.CardBody(
                [html.P(
                        "",
                        className="card-text",
                        id = 'right_estimate',
                        style = {'text-align':'center','color':'black'},
                    ),
                ]
            ),style = {'background-color':'#dfe3e6'}
        ),
    ], style = {'color':'white','background-color':'#FFFFFF','marginTop': 20, 'border':'2 px black solid'}
)


#preview_settings.style
import plotly.graph_objects as go
def preview_fig(image):
  # Create figure
  fig = go.Figure()

  # Constants
  img_width = 1080
  img_height = 360
  scale_factor = 1

  # Add invisible scatter trace.
  # This trace is added to help the autoresize logic work.
  fig.add_trace(
      go.Scatter(
          x=[0, img_width * scale_factor],
          y=[0, img_height * scale_factor],
          mode="markers",
          marker_opacity=0
      )
  )

  # Configure axes
  fig.update_xaxes(
      visible=False,
      range=[0, img_width * scale_factor]
  )

  fig.update_yaxes(
      visible=False,
      range=[0, img_height * scale_factor],
      # the scaleanchor attribute ensures that the aspect ratio stays constant
      scaleanchor="x"
  )
  # Add image
  fig.add_layout_image(
      dict(
          x=0,
          sizex=img_width * scale_factor,
          y=img_height * scale_factor,
          sizey=img_height * scale_factor,
          xref="x",
          yref="y",
          opacity=1.0,
          layer="below",
          sizing="stretch",
          source=PIL.Image.fromarray(image))
  )

  # Configure other layout
  fig.update_layout(
      width=img_width * scale_factor,
      height=img_height * scale_factor,
      margin={"l": 0, "r": 0, "t": 0, "b": 0},
  )
  return fig

def download_fig(image):
  # Create figure
  fig = go.Figure()

  # Constants
  img_width = 500
  img_height = 500
  scale_factor = 1

  # Add invisible scatter trace.
  # This trace is added to help the autoresize logic work.
  fig.add_trace(
      go.Scatter(
          x=[0, img_width * scale_factor],
          y=[0, img_height * scale_factor],
          mode="markers",
          marker_opacity=0
      )
  )

  # Configure axes
  fig.update_xaxes(
      visible=False,
      range=[0, img_width * scale_factor]
  )

  fig.update_yaxes(
      visible=False,
      range=[0, img_height * scale_factor],
      # the scaleanchor attribute ensures that the aspect ratio stays constant
      scaleanchor="x"
  )
  # Add image
  fig.add_layout_image(
      dict(
          x=0,
          sizex=img_width * scale_factor,
          y=img_height * scale_factor,
          sizey=img_height * scale_factor,
          xref="x",
          yref="y",
          opacity=1.0,
          layer="below",
          sizing="stretch",
          source=PIL.Image.fromarray(image))
  )

  # Configure other layout
  fig.update_layout(
      width=img_width * scale_factor,
      height=img_height * scale_factor,
      margin={"l": 0, "r": 0, "t": 0, "b": 0},
  )
  return fig

def find_results(n):
  results = []
  for folderName, subfolders, filenames in os.walk(df["generators"][0].dir["images"]):
    for filename in filenames:
        filePath = os.path.join(folderName, filename)
        results.append(filePath) 
  return [PIL.Image.open(result) for result in results[:n]]

def createImageGrid(images, scale=0.25, rows=1): #As array
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols)))
   return np.array(canvas)


##Stylegan settings

In [ ]:
stylegan_settings = dbc.Card(
        dbc.CardBody(
            [
                html.H3("Stylegan settings", className="card-title",style={'text-align':'center'}),
                   html.Label(html.H5(['Number of generated ',
                               html.Span(
                                'images:',
                                id='picture_description',
                                style={"textDecoration": "underline"}
                               )])),
                   
        
                  dbc.Tooltip('How many faces do you want to generate',target='picture_description'),



                  dbc.Input(id='number_of_images',type='number',
                            placeholder='Enter the number of images',
                            debounce= True, # zatwierdzi tylko jak naciśniemy enter lub zjedziemy kursorem obok )
                            min=1, max=1000, step=1,
                            size='570'),

                  html.Br(),


                    html.Label(html.H5([' The value of the ',
                              html.Span(
                              'truncation:',
                              id='truncation_description',
                              style={"textDecoration": "underline"}
                              )])),

                  ladowarka,

                  dbc.Tooltip('How varied should the generated faces be. A high value may produce unnatural faces (recommended value = 0.7)',target='truncation_description'),
                 


                 html.Div([
                  dcc.Slider(id='truncation',
                                min=0.0,
                                max=1.0,
                                step=0.05,
                                value=0.5,
                                persistence=True, #zachowa wartość po odświerzeniu strony
                                marks={
                                    0:{'label':'0.0','style': {'color': 'black'}},
                                    0.5:{'label':'0.5','style': {'color': 'black'}},
                                    1:{'label':'1.0','style': {'color': 'black'}}},
                                      ),
                        html.Div(id='slider-output-container1')]),
            ]
        ),style={"width": "34rem",'border':'2 px black solid','marginTop': 20},
    color='#1d9fad',   # https://bootswatch.com/default/ for more card colors
    inverse=True,   # change color of text (black or white)
    outline=False,  # True = remove the block colors from the background and header
)




##manipulation_settings

In [ ]:

manipulation_settings = dbc.Card(
    [
        dbc.CardBody([
            html.H3("Manipulation settings", className="card-title",style={'text-align':'center'}),
            html.Label(html.H5([html.Span(
                              'The manipulated trait',
                              id='manipulated_trait_description',
                              style={"textDecoration": "underline"}
                               ),":"])),

                       dbc.Tooltip('Which trait should be manipulated in the experimental conditions. Not all traits are experimentally validated!',target='manipulated_trait_description'),



                     dcc.Dropdown(id='which_trait', placeholder='Choose a trait',
                                  options=[
                                          {'label':'Age','value':'/content/psychoGAN/stylegan2/stylegan2directions/age.npy'},
                                          {'label':'Dominance','value':'/content/psychoGAN/stylegan2/stylegan2directions/dominance.npy'},
                                          {'label':'Gender','value':'/content/psychoGAN/stylegan2/stylegan2directions/gender.npy'},
                                          {'label':'Smile','value':'/content/psychoGAN/stylegan2/stylegan2directions/smile.npy'},
                                          {'label':'Trustworthiness','value':'/content/psychoGAN/stylegan2/stylegan2directions/trustworthiness.npy'}
                                           ],
                     multi=False,
                     value = None,
                     style={'width':'100%','color':'black'}),
                      
                      
                      html.Br(),
                      

#######################
                     html.Label(html.H5(['Manipulation strength:']),id='manipulation_description',
                                 style={'front-weight':'bold','text-align':'right',"textDecoration": "underline"}),
                      
 

                     dbc.Tooltip('How much manipulation should be applied. Values larger than 2 may be excessive for most use cases.',target='manipulation_description'),


                     html.Div([
                     dcc.Slider(id='manipulation_strength',
                                min=0,
                                max=2.5,
                                step=0.1,
                                value=1,
                                persistence=True, 
                                marks={
                                    0:{'label':'0','style': {'color': 'black'}},
                                    0.5:{'label':'0.5','style': {'color': 'black'}},
                                    1:{'label':'1','style': {'color': 'black'}},
                                    1.5:{'label':'1.5','style': {'color': 'black'}},
                                    2:{'label':'2','style': {'color': 'black'}},
                                    2.5:{'label':'2.5','style': {'color': 'black'}}}
                                      ),
                                    html.Div(id='slider-output-container')]),
                      html.Br(),


                      html.Label(html.H5(['Number of manipulation conditions:']),id='levels_description',
                                 style={'front-weight':'bold','text-align':'right',"textDecoration": "underline"}),

                      dbc.Tooltip('0 = no manipulation. 1 = like in the preview, generate every face with one manipulation to increase one to decrease. 2-3 Add additional, intermediate levels to the manipulation',target='levels_description'),

                      dbc.Input(id='number_of_levels',type='number',
                            placeholder='Enter the number of conditions',
                            debounce= True, # zatwierdzi tylko jak naciśniemy enter lub zjedziemy kursorem obok )
                            value = 0,
                            min=0, max=3, step=1,
                            size='570'),
        ])  
                     ],style={"width": "34rem",'border':'2 px black solid','marginTop': 20},
    color='#1d9fad',   # https://bootswatch.com/default/ for more card colors
    inverse=True,   # change color of text (black or white)
    outline=False,  # True = remove the block colors from the background and header

)

#Layouty poszczególnych stron

In [ ]:
### Strona główna z manpulation setting i stylegane setting
page_1_layout= dbc.Container([
              dcc.Store(id="store"),
    html.Div(id='hidden-div', style={'display': 'none'}),
                title,
                top_buttons,
                preview_card,
                preview_settings,
                dbc.Container (dbc.Row([   
                   stylegan_settings,
                  manipulation_settings
                                      ],justify='between'),
                 fluid=True),
                html.Iframe(id='console-out',srcDoc='',style={'backgroundColor':'#FFFFFF','width': '100%','height':400}),

            
              ])
            
        

### 2 strona 
page_2_layout = dbc.Container([
              
    html.Div(id='hidden-div', style={'display': 'none'}),
                title,

])


### 3 strona

page_3_layout = dbc.Container([
              
    html.Div(id='hidden-div', style={'display': 'none'}),
                title,    
     dbc.Row([        
       
            dbc.Card(
    dbc.CardBody(
        [ html.P("",id = "download-info"),
         dbc.Button("Download Generated Images", size="lg",id="btn_image",style={"width": 650}), 
         dcc.Download(id="download-image")])
    ,style={"position":"fixed", "right": 50, "bottom":55, "width": 690},
    #"position": "fixed", "bottom":55 , "right": 50,#"position": "fixed",'right':1000
    color='#1d9fad',  
    inverse=True,  
    outline=False,  ),
            #  html.Div([    dbc.Button("Download Image", size="lg",id="btn_image",style={"position": "fixed", "bottom":55 , "right": 50, "width": 650}), dcc.Download(id="download-image")]),  
         
          dbc.Card(
    dbc.CardBody(
       [ dcc.Graph(id = "download-graph", figure={})
        ])  #'right':1000 "position": "fixed"
    ,style={"width":"34rem"},
    color='#1d9fad',  
    inverse=True,  
    outline=False,  ),
     ],justify='between')
       
])

page_4_layout = dbc.Container([
              
    html.Div(id='hidden-div', style={'display': 'none'}),
                title,
    html.H2("Instruction", className="card-title",style={'text-align':'center', 'color': '#1d9fad'}),



        dbc.Row([    
                     


            dbc.Card(
    dbc.CardBody(
      instrukcja)
    ,style={"width":"34rem"},
    #"position": "fixed", "bottom":55 , "right": 50,#"position": "fixed",'right':1000
    color='#1d9fad',  
    inverse=True,
    outline=False,  ),



          dbc.Card(
    dbc.CardBody([
###
           html.Div(
    [
                dcc.Dropdown(id='ktory_film', placeholder='Choose a video preview',
                                  options=[
                                          {'label':'Dominance','value':'Dominance'},
                                          {'label':'Trustworthy','value':'Trust'},
                                          {'label':'Changing faces','value':'change_face'}
                                           ],
                     value = "Dominance",
                     style={'width':'1000','color':'black','right':'1000'}),
     
      
                     

    ]),                  
Filmy,
####
        ])  #'right':1000 "position": "fixed"
    ,style={"width":"34rem"},
    color='#1d9fad',  
    inverse=True,  
    outline=False,  ),
     ],justify='between')

])




##Włączanie apki

In [ ]:
app=JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, "https://codepen.io/chriddyp/pen/brPBPO.css"])
server = app.server
app.title = "Face generation"
df = pd.DataFrame([])
df["error"] = [""] 

all_w = [np.tile(np.array([1]*512),(18,1))]
image2 = {}

df["generators"]  = [None]

df["generators"][0] = generator(network_pkl="/content/psychoGAN/stylegan2/stylegan2-ffhq-config-f.pkl",
                              direction_name="Dominance", coefficient=1,
                              truncation=1, n_levels=3, n_photos=10, type_of_preview="manipulation",
                              result_dir="results", generator_number=1)
df["generators"][0].change_face()

graph = tf.get_default_graph()
sess = tf.get_default_session()

@app.callback(
     Output('slider-output-container', 'children'),
    [Input('manipulation_strength', 'drag_value'), Input('manipulation_strength', 'value')])
def update_output(drag_value,value):
    return   'drag value: {} | current value: {}'.format(drag_value, value)

@app.callback(
     Output('slider-output-container1', 'children'),
    [Input('truncation', 'drag_value'), Input('truncation', 'value')])
def update_output(drag_value,value):
    return   'drag value: {} | current value: {}'.format(drag_value, value)

#### Wczytywanie filmów

@app.callback(Output('film-content-classes', 'children'),
              [Input('ktory_film','value')],
              
              
              prevent_initial_call=False  )
def render_content(tab):
    if tab== 'change_face':
      return Film1_change_face
    elif tab== 'Dominance':
      return Film2_dominance
    elif tab=='Trust':
      return Film3_trust
    else: 
      return Film1_change_face




### Calback do pobierania twarzy, zakładaka download
@app.callback(
    [Output("download-image", "data"),
     Output("download-info","children")],
    Input("btn_image", "n_clicks"),
    prevent_initial_call=True,
)
def func(n_clicks):
  try:
    return dcc.send_file(
        '/content/psychoGAN/face_generation_results.zip'
    ), "Download successful!"
  except:
    return {}, "Please hit the GENERATE button and wait for the generation to finish"


@app.callback(
    Output("positioned-toast", "is_open"),
    [Input('preview', component_property="image")],
    [State("positioned-toast", "is_open")],
)
def open_toast(_, is_open):
    # if change_face == 0:
    #   return is_open
    return not is_open

########

@app.callback(
    [Output('preview',component_property='figure'),
     Output("change_face_button",component_property="n_clicks"),
     Output('left_estimate', component_property="children"),
     Output('right_estimate', component_property="children"),
     Output('preview-text', component_property="children")],
    [Input('which_trait','value'),
    Input('manipulation_strength','value'),
    Input('truncation','value'),
    Input("change_face_button","n_clicks")])   #Dodać null value handling
def update_information(direction_name, coefficient, truncation, change_face):
    global sess, graph
    with graph.as_default(), sess.as_default():
      # df["error"][0] = str(df["generators"][0].Gs.components)+"\n        "
      df["generators"][0].coefficient = coefficient
      df["generators"][0].truncation = truncation
      if direction_name is None:
        df["generators"][0].type_of_preview = "3_faces"  
      else:
        df["generators"][0].type_of_preview = "manipulation"
        df["generators"][0].direction_name = direction_name
      df["generators"][0].coefficient = coefficient
      if change_face:
        df["generators"][0].change_face()
      
      image2  =  df["generators"][0].refresh_preview()
      image2 = preview_fig(image2)

    if direction_name is None:
      return image2, 0, "-", "-", "Choose a trait for manipulation"
    elif "dominance" in direction_name:
      return image2, 0, "{:.2f} SD Less Dominant".format(-coefficient*0.47805/0.7764), "+{:.2f} SD More Dominant".format(coefficient*0.47805/0.7764), "The estimated difference in judgements in terms of rating SDs"
    elif "trustworthiness" in direction_name:
      return image2, 0, "{:.2f} SD Less Trustworthy".format((-0.42107432*coefficient  + 0.0624*coefficient**2)/0.7592), "+{:.2f} SD More Trustworthy".format((0.42107432 * coefficient -0.09403035*coefficient**2)/0.7592), "The estimated difference in judgements in terms of rating SDs"
    else:
      return image2, 0, "Decreased level of the trait", "Increased level of the trait", "Warning: The accuracy of this manipulation has not been checked experimentally"
    
##Debugger: feedback konsoli na dole strony, odkomentować dla trybu debugowania
# @app.callback(Output('console-out', 'srcDoc'),
#     [Input(component_id='truncation',component_property='value'),
#      Input("store", "data")])
# def update_output(n,_):
#     return "Callback error: \n"+str(df["error"][0])

#################################   Guzik generate z oknem ładowania
@app.callback(
    Output("modal", "is_open"),
    [Input("generate_button", "n_clicks"),
      Input("cancel", "n_clicks")],
    [State("modal", "is_open")]
)
def toggle_modal(n1, n2, is_open):
  if n1 or n2:
    return not is_open
      

#################################  Sam proces generacji
@app.callback(
    Output("header-generation", "children"),
    [Input("modal", "is_open")],#, Input("close", "n_clicks")],
    [State('which_trait','value'),
     State('manipulation_strength','value'),
     State('truncation','value'),
     State('number_of_levels','value'),
     State('number_of_images','value'),
     ]
)
def generation(is_open, direction_name, manipulation_strength,truncation,number_of_levels,number_of_images): 
  if is_open:
    try:
      global sess, graph
      with graph.as_default(), sess.as_default():
        df["generators"][0].coefficient=manipulation_strength
        df["generators"][0].truncation=truncation
        if direction_name is not None and ("dominance" in direction_name or "trust" in direction_name):
          df["generators"][0].direction_name = direction_name.split("/")[-1].replace(".npy",'')
        elif direction_name is not None:
          df["generators"][0].direction_name = direction_name
        df["generators"][0].n_levels=number_of_levels
        df["generators"][0].n_photos= number_of_images
        df["generators"][0].generate()
              
    except Exception as e:
      #TUTAJ DODJMY JAKIŚ RETURN KTÓRY INFORMUJE ŻE NIE WSZYSTKIE USTAWIENIA ZOSTAŁY WYBRANE
      df["error"][0] +=  str(e)+"\n      "
      df["error"][0] +=  str(traceback.format_exc())
      return "Set all the settings before generation"
    # if len(os.listdir('/content/psychoGAN/results1/images') ) == number_of_images*(number_of_levels*2):
    return "Generation succesful! Download your stimuli"


app.layout = html.Div([
    dcc.Tabs(
        id="tabs-with-classes",
        value='tab-1',
        parent_className='custom-tabs',
        className='custom-tabs-container',
        children=[
            dcc.Tab(
                label='Generate',
                value='tab-1',
                className='custom-tab',
                selected_className='custom-tab--selected'
            ),
            # dcc.Tab(
            #     label='Clean up',
            #     value='tab-2',
            #     className='custom-tab',
            #     selected_className='custom-tab--selected'
            # ),
            dcc.Tab(
                label='Download',
                value='tab-3', className='custom-tab',
                selected_className='custom-tab--selected'
            ),
              dcc.Tab(
                label='Instruction',
                value='tab-4', className='custom-tab',
                selected_className='custom-tab--selected',
                #style={'color':'#f9f9f9'}
            ),
        ]),
    html.Div(id='tabs-content-classes')
])

@app.callback(Output("download-graph","figure"),
              Input('tabs-with-classes', 'value')   
)
def download_graph_load(tab):
  return download_fig(createImageGrid(find_results(9), rows=3))

@app.callback(Output('tabs-content-classes', 'children'),
              Input('tabs-with-classes', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return page_1_layout
        
    elif tab == 'tab-2':
        return page_2_layout
        
    elif tab == 'tab-3':
        return page_3_layout
    elif tab == 'tab-4':
        return page_4_layout
    else:
      return page_1_layout


app.run_server(debug=False, mode='external')

Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2021 09:00:04] "GET /_alive_c08335fa-04e1-4d4a-859c-c77b86c21372 HTTP/1.1" 200 -


Dash app running on:


<IPython.core.display.Javascript object>